In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [51]:

engine = create_engine("sqlite:///./poc.db") 
df = pd.read_sql("SELECT * FROM hired_employees", engine)
print(df)

        id                name              datetime  department_id  job_id
0        1         Harold Vogt  2021-11-07T02:48:42Z              2      96
1        3         Lyman Hadye  2021-09-01T23:27:38Z              5      52
2        4       Lotti Crowthe  2021-10-01T13:04:21Z             12      71
3        5      Gretna Lording  2021-10-10T22:22:17Z              6      80
4        6      Marlow Antecki  2021-04-23T23:45:42Z              6      95
...    ...                 ...                   ...            ...     ...
1924  1995          Goran Yong  2021-08-22T21:57:53Z              2      83
1925  1996  Cirstoforo Youings  2021-04-01T17:48:42Z              3      23
1926  1997     Wilek Yurkevich  2021-05-04T12:19:50Z              2     132
1927  1998          Jerry Yven  2021-10-03T14:12:50Z              7     100
1928  1999       Jerri Zebedee  2022-01-18T10:47:37Z              8      80

[1929 rows x 5 columns]


In [3]:
# Number of employees hired for each job and department in 2021 divided by quarter. The
# table must be ordered alphabetically by department and job.

datefrom = '2021-01-01'
dateto = '2021-12-31'

engine = create_engine("sqlite:///./poc.db") 

from sqlalchemy import text

# Sqlite does not support UNPIVOT function, but in this way it is simulated
query = text("""
    SELECT department Department, job Job, 
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 1 THEN 1 ELSE 0 END) AS Q1,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 2 THEN 1 ELSE 0 END) AS Q2,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 3 THEN 1 ELSE 0 END) AS Q3,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 4 THEN 1 ELSE 0 END) AS Q4
    FROM hired_employees e
    LEFT JOIN jobs j
    ON e.job_id = j.id
    LEFT JOIN departments d
    ON e.department_id = d.id
    WHERE datetime >= :datefrom AND datetime <= :dateto
    GROUP BY department, job
    ORDER BY department, job
""")

df = pd.read_sql(query, con=engine, params={
    "datefrom": f"{datefrom}T00:00:00Z",
    "dateto": f"{dateto}T23:59:59Z"
})

display(df)

,Department,Job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
...,...,...,...,...,...,...
928,Training,Teacher,0,2,0,0
929,Training,Technical Writer,0,0,1,0
930,Training,VP Product Management,1,0,0,0
931,Training,VP Quality Control,0,1,0,0


In [45]:
# List of ids, name and number of employees hired of each department that hired more
# employees than the mean of employees hired in 2021 for all the departments, ordered
# by the number of employees hired (descending).

datefrom = '2021-01-01'
dateto = '2021-12-31'

engine = create_engine("sqlite:///./poc.db") 

from sqlalchemy import text

query = text("""
    SELECT d.department Department, count(*) Hired
    FROM hired_employees e
    INNER JOIN (
                SELECT *
                FROM (         
                        SELECT department_id, count(*) hires_dep,
                        AVG(COUNT(*)) OVER () AS avg_hires
                        FROM hired_employees e
                        WHERE datetime >= :datefrom AND datetime <= :dateto
                        GROUP BY department_id
                    )q
                WHERE hires_dep > avg_hires             
             ) deps_hire_over_mean         
    ON deps_hire_over_mean.department_id = e.department_id
    LEFT JOIN departments d
    ON d.id = e.department_id
    GROUP BY e.department_id, d.department
    ORDER BY Hired desc
""")

df = pd.read_sql(query, con=engine, params={
    "datefrom": f"{datefrom}T00:00:00Z",
    "dateto": f"{dateto}T23:59:59Z"
})

display(df)

,Department,Hired
0,Support,248
1,Human Resources,242
2,Engineering,241
3,Services,233
4,Business Development,215
5,Research and Development,173
6,Marketing,164


In [57]:

engine = create_engine("sqlite:///./poc.db") 
engine.execute("delete from jobs")
